# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [1]:
#!pip install -qU ragas==0.2.10

In [2]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

### NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [3]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /home/ookami/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ookami/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [4]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [5]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [6]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - which should hopefull be familiar at this point since it's our Loan Data use-case!

Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [7]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyMuPDFLoader


path = "data/"
loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [8]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

/home/ookami/ml/AIE/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/segmenter.py:66: SyntaxWarning: invalid escape sequence '\s'
  for match in re.finditer('{0}\s*'.format(re.escape(sent)), self.original_text):
/home/ookami/ml/AIE/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/lang/arabic.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)
/home/ookami/ml/AIE/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/lang/persian.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)


Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [9]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [10]:
from ragas.testset.graph import Node, NodeType

### NOTICE: We're using a subset of the data for this example - this is to keep costs/time down.
for doc in docs[:20]:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 20, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [11]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/17 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/20 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/31 [00:00<?, ?it/s]

Property 'summary' already exists in node '70f84c'. Skipping!
Property 'summary' already exists in node 'b73d67'. Skipping!
Property 'summary' already exists in node 'a97ebd'. Skipping!
Property 'summary' already exists in node '73aca6'. Skipping!
Property 'summary' already exists in node '2db51e'. Skipping!
Property 'summary' already exists in node 'c50c28'. Skipping!
Property 'summary' already exists in node '5e2dc4'. Skipping!
Property 'summary' already exists in node 'b8ee37'. Skipping!
Property 'summary' already exists in node '3ea63f'. Skipping!
Property 'summary' already exists in node 'bea5c0'. Skipping!
Property 'summary' already exists in node 'b3c258'. Skipping!
Property 'summary' already exists in node '9fd6f4'. Skipping!
Property 'summary' already exists in node 'a406c2'. Skipping!
Property 'summary' already exists in node '6433af'. Skipping!


Applying CustomNodeFilter:   0%|          | 0/6 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/43 [00:00<?, ?it/s]

Property 'summary_embedding' already exists in node '70f84c'. Skipping!
Property 'summary_embedding' already exists in node 'b73d67'. Skipping!
Property 'summary_embedding' already exists in node '3ea63f'. Skipping!
Property 'summary_embedding' already exists in node '2db51e'. Skipping!
Property 'summary_embedding' already exists in node '5e2dc4'. Skipping!
Property 'summary_embedding' already exists in node '9fd6f4'. Skipping!
Property 'summary_embedding' already exists in node 'a97ebd'. Skipping!
Property 'summary_embedding' already exists in node 'bea5c0'. Skipping!
Property 'summary_embedding' already exists in node 'c50c28'. Skipping!
Property 'summary_embedding' already exists in node 'b8ee37'. Skipping!
Property 'summary_embedding' already exists in node 'a406c2'. Skipping!
Property 'summary_embedding' already exists in node '73aca6'. Skipping!
Property 'summary_embedding' already exists in node 'b3c258'. Skipping!
Property 'summary_embedding' already exists in node '6433af'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 40, relationships: 478)

We can save and load our knowledge graphs as follows.

In [12]:
kg.save("loan_data_kg.json")
loan_data_kg = KnowledgeGraph.load("loan_data_kg.json")
loan_data_kg

KnowledgeGraph(nodes: 40, relationships: 478)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [13]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=loan_data_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [14]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.

`SingleHopSpecific:` a question that requires a single contextual answer from a single document

`MultiHopSpecific:` a question that's specific to a single document, but whose complete comprehension/answer requires hops to multiple documents - similar to asking for a quote from the top-rated book in the year 1997; the model would need to first find a doc thta shows the rank of books in 1997, and from there find the book name/author and search the doc for that book to find the relevant quote

`MultiHopAbstract:` a question that requires accessing multiple documents to form a conceptual understanding to answer a question - similar to posing a research question and the model needing to access relevant documents to make its own answer of how it may be possible to achieve what's being researched


Finally, we can use our `TestSetGenerator` to generate our testset!

In [15]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,What is the Knoledge Center?,"[Chapter 1 Academic Years, Academic Calendars,...",The context does not provide a definition or d...,single_hop_specifc_query_synthesizer
1,What is 34 CFR 668 3 a about in academic year ...,[Regulatory Citations Academic year minimums: ...,34 CFR 668.3(a) relates to the minimum weeks o...,single_hop_specifc_query_synthesizer
2,Chapter 3 is it like a standard term or not be...,[Inclusion of Clinical Work in a Standard Term...,Inclusion of clinical work in a standard term ...,single_hop_specifc_query_synthesizer
3,Is the FWS program considered a payment period...,[Non-Term Characteristics A program that measu...,The payment period is applicable to all Title ...,single_hop_specifc_query_synthesizer
4,Chapters 5 and 6 what are they in the context ...,[both the credit or clock hours and the weeks ...,The context discusses how scheduled payment pe...,single_hop_specifc_query_synthesizer
5,Considering the requirements for participation...,[<1-hop>\n\nInclusion of Clinical Work in a St...,The inclusion of clinical work in standard ter...,multi_hop_abstract_query_synthesizer
6,Participation and requirements for practicum o...,[<1-hop>\n\nInclusion of Clinical Work in a St...,The context explains that clinical work includ...,multi_hop_abstract_query_synthesizer
7,How does the impact of term length and measure...,[<1-hop>\n\nInclusion of Clinical Work in a St...,The inclusion of clinical work in standard ter...,multi_hop_abstract_query_synthesizer
8,"In volume 8 and volume 7, how clinical work in...",[<1-hop>\n\nInclusion of Clinical Work in a St...,The context explains that clinical work includ...,multi_hop_specific_query_synthesizer
9,"In Volume 2 and Volume 8, how does the academi...","[<1-hop>\n\nChapter 1 Academic Years, Academic...","According to Volume 2, the academic year must ...",multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [16]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs[:20], testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/17 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/20 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/31 [00:00<?, ?it/s]

Property 'summary' already exists in node '55fb4f'. Skipping!
Property 'summary' already exists in node 'bfdff7'. Skipping!
Property 'summary' already exists in node '40c111'. Skipping!
Property 'summary' already exists in node '2e1dcf'. Skipping!
Property 'summary' already exists in node '143ad4'. Skipping!
Property 'summary' already exists in node 'b946bf'. Skipping!
Property 'summary' already exists in node 'fc9fe1'. Skipping!
Property 'summary' already exists in node '08a9b3'. Skipping!
Property 'summary' already exists in node 'cd779c'. Skipping!
Property 'summary' already exists in node '5daa1b'. Skipping!
Property 'summary' already exists in node '7266f5'. Skipping!
Property 'summary' already exists in node '814c20'. Skipping!
Property 'summary' already exists in node 'f51f81'. Skipping!
Property 'summary' already exists in node '9c5f9e'. Skipping!


...

Property 'summary_embedding' already exists in node '40c111'. Skipping!
Property 'summary_embedding' already exists in node '55fb4f'. Skipping!
Property 'summary_embedding' already exists in node 'f51f81'. Skipping!
Property 'summary_embedding' already exists in node 'fc9fe1'. Skipping!
Property 'summary_embedding' already exists in node 'cd779c'. Skipping!
Property 'summary_embedding' already exists in node '2e1dcf'. Skipping!
Property 'summary_embedding' already exists in node '143ad4'. Skipping!
Property 'summary_embedding' already exists in node '5daa1b'. Skipping!
Property 'summary_embedding' already exists in node '7266f5'. Skipping!
Property 'summary_embedding' already exists in node 'b946bf'. Skipping!
Property 'summary_embedding' already exists in node '08a9b3'. Skipping!
Property 'summary_embedding' already exists in node 'bfdff7'. Skipping!
Property 'summary_embedding' already exists in node '814c20'. Skipping!
Property 'summary_embedding' already exists in node '9c5f9e'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [17]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,What is the role of the School Participation D...,"[Chapter 1 Academic Years, Academic Calendars,...",The School Participation Division oversees com...,single_hop_specifc_query_synthesizer
1,Could you please explain the significance of t...,[Regulatory Citations Academic year minimums: ...,Regulatory citation 34 CFR 668.3(a) pertains t...,single_hop_specifc_query_synthesizer
2,Volume 8 is it a standard term?,[Inclusion of Clinical Work in a Standard Term...,Inclusion of clinical work in a standard term ...,single_hop_specifc_query_synthesizer
3,What is Title IV in relation to payment periods?,[Non-Term Characteristics A program that measu...,"Title IV program disbursements, except for the...",single_hop_specifc_query_synthesizer
4,How do accelerated progression effects influen...,[<1-hop>\n\nboth the credit or clock hours and...,Accelerated progression in clock-hour or non-t...,multi_hop_abstract_query_synthesizer
5,How do the credit hour completion requirements...,[<1-hop>\n\nboth the credit or clock hours and...,The context explains that disbursement of Pell...,multi_hop_abstract_query_synthesizer
6,How do the regulatory citations 34 CFR 668.3(a...,"[<1-hop>\n\nChapter 1 Academic Years, Academic...",The regulatory citations 34 CFR 668.3(a) and 3...,multi_hop_abstract_query_synthesizer
7,disbursement timing in subscription programs a...,[<1-hop>\n\nboth the credit or clock hours and...,"In the context of subscription-based programs,...",multi_hop_abstract_query_synthesizer
8,so in chapter 2 and chapter 3 how do they like...,"[<1-hop>\n\nChapter 1 Academic Years, Academic...",Chapter 2 explains that academic years must in...,multi_hop_specific_query_synthesizer
9,How do Appendix A and Appendix B relate to the...,[<1-hop>\n\nDisbursement Timing in Subscriptio...,Appendix B provides detailed guidance on disbu...,multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [18]:
from langsmith import Client

client = Client()

dataset_name = "Loan Synthetic Data"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Loan Synthetic Data"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [19]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [20]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [21]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [22]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [23]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Loan RAG"
)

In [24]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [25]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [26]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

Finally, we can set-up our RAG LCEL chain!

In [27]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [28]:
rag_chain.invoke({"question" : "What kinds of loans are available?"})

'The kinds of loans available include:\n\n- Direct Subsidized Loans  \n- Direct Unsubsidized Loans  \n- Direct PLUS Loans (student Federal PLUS Loans and parent Direct PLUS Loans)  \n- Subsidized Federal Stafford Loans  \n- Unsubsidized Federal Stafford Loans  \n- Federal SLS Loans  \n- Federal PLUS Loans (made under the Federal Family Education Loan (FFEL) Program before July 1, 2010)\n\nAdditionally, for graduate or professional students, only Direct Unsubsidized Loans and Direct PLUS Loans are available, as graduate or professional students are not eligible for Direct Subsidized Loans.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [29]:
eval_llm = ChatOpenAI(model="gpt-4.1")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [30]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

empathy_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "empathy": "Is this response empathetic? Does it make the user feel like they are being heard?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.


`qa_evaluator`: evaluates correctness of the answer by comparing the model's response to the expected answer. It checks if the response accurately answers the question based on factual correctness.

`labeled_helpfulness_evaluator`: evaluates the helpfulness of the response by considering both the model's output and reference answer. It assesses whether the response is useful and beneficial to the user, taking into account what the correct answer should be.

`empathy_evaluator`: evaluates the response's empathy by assessing whether it demonstrates understanding and makes the user feel heard. It focuses on the emotional tone and user experience instead of factual accuracy.

## LangSmith Evaluation

In [31]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'earnest-pest-76' at:
https://smith.langchain.com/o/7b0ecff6-e086-4031-830d-c4302b07d6ad/datasets/5b4e586b-4b03-470d-beb0-6cf9e5e91f82/compare?selectedSessions=d5e04f52-2f2d-4544-b617-53989224433a




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,How do Volume 8 and Volume 7 relate to inclusi...,"Based on the context provided:\n\n- Volume 7, ...",None,Volume 8 provides guidance on including clinic...,1,1,0,9.151645,1151c6d4-74af-4543-a6ec-aa2e30554f7b,6613714f-c054-4637-bb4c-894ff7315b97
1,Considering the detailed requirements for defi...,"Based on the provided context, the definition ...",None,The definition of academic year and instructio...,1,1,0,12.244229,ea86198a-fb17-4fad-99f0-f3a57a6d041b,122ec69f-99f8-4fc3-b14f-2aceda0abe3d
2,How do Appendix A and Appendix B relate to the...,"Appendix A in Volume 3, Chapter 1 provides exa...",None,Appendix B provides detailed guidance on disbu...,1,1,0,3.724050,a08a6383-b4fe-4a81-968c-f0f93e8423ea,67ff48be-e5c2-44ee-9561-1baf762712db
3,so in chapter 2 and chapter 3 how do they like...,Based on the provided context:\n\nChapter 2 di...,None,Chapter 2 explains that academic years must in...,1,1,0,9.741440,f11caee2-b984-4ab6-b8dc-85ff6ab01d40,34642136-6138-47e2-8602-6045271b65b3
4,disbursement timing in subscription programs a...,"For subscription-based programs, the disbursem...",None,"In the context of subscription-based programs,...",1,1,0,7.684623,074c73de-44a8-4f33-9813-8f81e6bf1a4a,3319380b-4e39-40de-a6ea-0bca1ceca4ec
5,How do the regulatory citations 34 CFR 668.3(a...,The regulatory citations 34 CFR 668.3(a) and 3...,None,The regulatory citations 34 CFR 668.3(a) and 3...,1,1,0,2.898691,da5f88ca-bee3-4b09-ba76-b87e839782d9,c40f74dc-f6e8-4b26-893d-d3dd35e51781
6,How do the credit hour completion requirements...,"Based on the provided context, the Pell Grant ...",None,The context explains that disbursement of Pell...,1,1,0,6.006703,b21b9f84-9b0d-40a7-ad96-1763ff040c4c,fb09e34b-4fb4-42d9-8e16-6e7c516d2ffc
7,How do accelerated progression effects influen...,"Based on the provided context, accelerated pro...",None,Accelerated progression in clock-hour or non-t...,1,0,0,5.062648,b0f82553-f82a-466e-918c-df30c41eea0c,4a8f04d7-961c-4dce-88c5-744564183699
8,What is Title IV in relation to payment periods?,Title IV refers to federal student financial a...,None,"Title IV program disbursements, except for the...",1,1,0,2.037239,9d735a76-3994-4fff-813b-254f85872579,02e5c944-9b6f-430f-9270-d75c7fbc5e34
9,Volume 8 is it a standard term?,I don't know.,None,Inclusion of clinical work in a standard term ...,1,0,0,0.578487,56a62a36-5c62-4ea0-b50e-b31ebe49632c,86abf173-58ee-4b4a-b61b-e69d09da9bcb


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [32]:
EMPATHY_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the question using empathy and kindness, and make sure the user feels heard.

Context: {context}
Question: {question}
"""

empathy_rag_prompt = ChatPromptTemplate.from_template(EMPATHY_RAG_PROMPT)

In [33]:
rag_documents = docs

In [34]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

```
Chunk size would include more context into the embedding. This could both hurt and benefit certain context similarity.

1) It could help if increasing the chunk allows more related context into chunk, better helping its similarity score
2) It could hurt if increasing the chunk dilutes the chunk by letting in too much unrelated context
```

In [35]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

`Because it allows for a larger dimensionality, in theory, it's able to capture more nuanced semantic relationships, not only more precise similarities, but bring about different aspects of what it means to be "similar"`

In [36]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Loan Data for RAG"
)

In [37]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [38]:
empathy_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | empathy_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [39]:
empathy_rag_chain.invoke({"question" : "What kinds of loans are available?"})

"Thank you for your question—it's great that you're seeking clarity on the types of loans available, as understanding your options can really help ease the process. Based on the information provided, there are several types of loans available:\n\n1. **Direct Subsidized Loans:** These loans are based on financial need, and the government pays the interest while you're in school at least half-time, during the grace period, and during deferment.\n\n2. **Direct Unsubsidized Loans:** These loans are not based on financial need, and interest accrues during all periods, including while you're in school.\n\n3. **Direct PLUS Loans:** These loans are available to parents of dependent students (Direct PLUS for parents) or to graduate/professional students (Direct PLUS Loans for students). They can help cover the student's cost of attendance but require the borrower to meet eligibility criteria, including credit approval.\n\nIt's also mentioned that a dependent student whose parent is unable to ob

Finally, we can evaluate the new chain on the same test set!

In [40]:
evaluate(
    empathy_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "empathy_rag_chain"},
)

View the evaluation results for experiment: 'puzzled-yard-21' at:
https://smith.langchain.com/o/7b0ecff6-e086-4031-830d-c4302b07d6ad/datasets/5b4e586b-4b03-470d-beb0-6cf9e5e91f82/compare?selectedSessions=adf8d944-52d2-4454-946d-485932793d09




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,How do Volume 8 and Volume 7 relate to inclusi...,Thank you for your thoughtful question. It’s c...,None,Volume 8 provides guidance on including clinic...,0,0,1,6.229143,1151c6d4-74af-4543-a6ec-aa2e30554f7b,03f92bc8-574d-4ab2-8d1f-d23567749a54
1,Considering the detailed requirements for defi...,Thank you for your thoughtful question. It’s c...,None,The definition of academic year and instructio...,1,1,1,9.530904,ea86198a-fb17-4fad-99f0-f3a57a6d041b,8e441f2d-453c-4256-9196-d872be00c69a
2,How do Appendix A and Appendix B relate to the...,Thank you for your thoughtful question. Based ...,None,Appendix B provides detailed guidance on disbu...,0,0,1,4.004248,a08a6383-b4fe-4a81-968c-f0f93e8423ea,08fead57-ed80-4579-a9e9-2acc609654e4
3,so in chapter 2 and chapter 3 how do they like...,Thank you for your thoughtful question. I can ...,None,Chapter 2 explains that academic years must in...,1,1,1,5.685667,f11caee2-b984-4ab6-b8dc-85ff6ab01d40,e7607adb-eff5-45b8-8148-4b48fd850909
4,disbursement timing in subscription programs a...,Thank you for your thoughtful question about d...,None,"In the context of subscription-based programs,...",1,1,1,5.673759,074c73de-44a8-4f33-9813-8f81e6bf1a4a,54369736-8a45-4e46-93a0-b8f9e888b200
5,How do the regulatory citations 34 CFR 668.3(a...,Thank you for your thoughtful question. Based ...,None,The regulatory citations 34 CFR 668.3(a) and 3...,1,0,1,3.879986,da5f88ca-bee3-4b09-ba76-b87e839782d9,0534e2b4-6526-4d3f-bbfc-6f3878097520
6,How do the credit hour completion requirements...,Thank you for your thoughtful question. Based ...,None,The context explains that disbursement of Pell...,1,0,1,4.868651,b21b9f84-9b0d-40a7-ad96-1763ff040c4c,9572788c-8f38-4772-bb8d-0fff28a8fd18
7,How do accelerated progression effects influen...,I can see you're trying to understand somethin...,None,Accelerated progression in clock-hour or non-t...,0,0,1,4.867177,b0f82553-f82a-466e-918c-df30c41eea0c,49982cb8-52a8-48f0-a2e1-a1459f549bc7
8,What is Title IV in relation to payment periods?,Thank you for your question—it's important to ...,None,"Title IV program disbursements, except for the...",1,1,1,2.868380,9d735a76-3994-4fff-813b-254f85872579,a3cc82f3-1d57-4411-9197-a0e242feda18
9,Volume 8 is it a standard term?,Thank you for your question. Based on the cont...,None,Inclusion of clinical work in a standard term ...,1,1,1,3.545333,56a62a36-5c62-4ea0-b50e-b31ebe49632c,5c3f21d4-a98b-42a2-b4dd-2b19e0809dee


#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.

```
So the most obvious metric change is empathy. Because we explicity added to the prompt to be more emphatic, and because it's unrelated in anyway to the actual context of the RAG retrieval, marks went up all around and everything that failed previously, now pass.

Latency has also gone up, likely do to a few things. 
1) the embedding dimensions are now larger, likely contributing to longer retrieval times
2) adding the empathy requirements likely adds additional processing by the LLM to ensure its responses fit the request

What may surprise most, is the drop in correctness and helpfulness, despite the fact that we're now also using a better text embedder. I'm almost positive that the cause is due to the massive 2x increase in chunk size. This most likely allows way too much irrelevant data, making similarities more "normalized", and thus the scores with a higher similarity might not actually be the most relevant. Because there's a lot of contextually useless information now, the larger text embedder doesn't know which nuances to focus on to increase similarity, since it can't distinguish what aspects of the chunk are actually what's desired. An analogy could be as follows: it's like trying to give help to "Waldo" in where's Waldo - is it easier to find and help Waldo when there's a ton going on and many people that look like him? Or when there's less people in general and less people that look similar to him?


```

<div style="display: flex; width: 100%;">
    <img src="/home/ookami/ml/AIE/07_Synthetic_Data_Generation_and_LangSmith/text-small.png" alt="Image 1" style="width: 50%; margin-right: 10px;">
    <img src="/home/ookami/ml/AIE/07_Synthetic_Data_Generation_and_LangSmith/text-large.png" alt="Image 2" style="width: 50%; margin-left: 10px;">
  </div>
